In [1]:
import pandas as pd
import backtrader as bt
from datetime import datetime
from loguru import logger

In [4]:
class CrossOverPercentStop(bt.Strategy):
    # параметры для скользящих средних
    params = dict(
        fast_ma=10,
        slow_ma=20,
        stop_loss=2
    )

    def __init__(self):
        '''рассчёт скользящих средних и индикатора их пересечения'''
        fast_ma = bt.ind.SMA(period=self.p.fast_ma)
        slow_ma = bt.ind.SMA(period=self.p.slow_ma)
        self.crossover = bt.ind.CrossOver(fast_ma, slow_ma)
        
        self.orderl = list() # для хранения состояния заявки

    def notify_order(self, order):
        '''вывод сделок в консоль'''
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                logger.info(f'BUY@price {order.executed.price}')
            elif order.issell():
                logger.info(f'SELL@price {order.executed.price}')
        elif order.status in [order.Margin, order.Rejected]:
            logger.warning('Order Margin/Rejected')

        self.order = None

    def next(self):
        if not self.position and self.crossover > 0:
                stop_price = self.data.close[0] * (1.0 - self.p.stop_loss/100)
                o1 = self.buy(transmit=False) # основная зявки
                o2 = self.sell(exectype=bt.Order.Stop, price=stop_price, parent=o1, transmit=True) # стоп
                self.orderl = [o1, o2]
        elif self.crossover < 0:
            for o in self.orderl: 
                self.cancel(o)
            self.close()


In [6]:
cerebro = bt.Cerebro()
commission = 0.05
cerebro.broker.setcommission(commission=commission/100, leverage=1)
cerebro.broker.setcash(10000.) # стартовый кэш
cerebro.addsizer(bt.sizers.PercentSizer, percents=95) # размер позиции

df = pd.read_csv(
    '..//data//1d//AAPL.csv', # файл с котировками
    index_col=0,
    parse_dates=True
)

data = bt.feeds.PandasData(
    dataname=df,
    openinterest=-1,
    fromdate=datetime(2005, 1, 1), # стартовая дата
    # todate=datetime(2005, 2, 1),
    timeframe=bt.TimeFrame.Days,
    compression=1
)

cerebro.adddata(data) # добавляем котировки
cerebro.addstrategy(CrossOverPercentStop,fast_ma=10,slow_ma=20,stop_loss=10) # добавляем стратегию
cerebro.run() # запускаем рассчёты

2021-10-13 15:51:39.491 | INFO     | __main__:notify_order:24 - BUY@price 1.5199999809265134
2021-10-13 15:51:39.506 | INFO     | __main__:notify_order:26 - SELL@price 1.5175000429153442
2021-10-13 15:51:39.521 | INFO     | __main__:notify_order:24 - BUY@price 1.264999985694885
2021-10-13 15:51:39.527 | INFO     | __main__:notify_order:26 - SELL@price 1.2342859506607056
2021-10-13 15:51:39.539 | INFO     | __main__:notify_order:24 - BUY@price 1.4107140302658079
2021-10-13 15:51:39.547 | INFO     | __main__:notify_order:26 - SELL@price 1.276071345806122
2021-10-13 15:51:39.574 | INFO     | __main__:notify_order:24 - BUY@price 1.3296430110931396
2021-10-13 15:51:39.590 | INFO     | __main__:notify_order:26 - SELL@price 1.3703570365905764
2021-10-13 15:51:39.608 | INFO     | __main__:notify_order:24 - BUY@price 1.4632140398025515
2021-10-13 15:51:39.640 | INFO     | __main__:notify_order:26 - SELL@price 1.5521429777145386
2021-10-13 15:51:39.648 | INFO     | __main__:notify_order:24 - BUY

In [7]:
cerebro.plot(iplot=False) # рисуем графики

[[<Figure size 1141x444 with 5 Axes>]]